# Problem Set 7
## Daryl Larsen
### 3. (Monte Carlo simulations for Probit)

In [1]:
using LinearAlgebra, Random, Distributions, PrettyTables, Optim, Plots

#### (a) Generate data

In [2]:
# Set some seed (π sounds good)
Random.seed!(314159265358)

function data(n; β=[-0.5 , 1], ρ=0.9, δ=[0 0])
    # generate standard normal X and ones
    X1 = ones(n, 1)
    X2 = rand(Normal(0,1),n)
    X = [X1 X2]
    
    # generate standard normal u and Y according to formula
    U = rand(Normal(0,1),n)
    Y = 1 .* (X*β .+ U .>= 0)
    
    return ( Y = Y, X = X, U = U)  
end

data (generic function with 1 method)

#### (b) Construct sample criterion function

In [3]:
function sampleCriterion(n,Y,b,X)
    xb = X * b
    Φ = zeros(n,1)
    Φ1 = zeros(n,1)
    for j = 1:n
        Φ[j,1] = log(cdf(Normal(0,1),xb[j,1]))
        Φ1[j,1] = log(1 - cdf(Normal(0,1),xb[j,1]))
    end
    Q = - (Y' * Φ) ./n - ((1 .- Y)' * Φ1) ./n
    
    return Q
end 

sampleCriterion (generic function with 1 method)

#### (c) Compute MLE of β0

In [4]:
# generate data
n = 100
(Y, X, U) = data(n)

result = optimize(b->sampleCriterion(n,Y,b,X)[1,1],[0.0 ; 0.0])
βprobit = Optim.minimizer(result)

2-element Vector{Float64}:
 -0.5881186381151735
  1.0361748198847573

#### (d) Logit criterion function

In [5]:
function logitCriterion(n,Y,b,X)
    xb = X * b
    Λ = zeros(n,1)
    Λ1 = zeros(n,1)
    for j = 1:n
        Λ[j,1] = log( exp(xb[j,1]) ./ (1 .+ exp(xb[j,1])))
        Λ1[j,1] = log(1 - exp(xb[j,1]) ./ (1 .+ exp(xb[j,1])))
    end
    Q = - (Y' * Λ) ./n - ((1 .- Y)' * Λ1) ./n
    
    return Q
end

logitCriterion (generic function with 1 method)

In [6]:
result = optimize(b->logitCriterion(n,Y,b,X)[1,1],[0.0 ; 0.0])
βlogit = Optim.minimizer(result)

2-element Vector{Float64}:
 -1.0159651032899697
  1.8127426924132513

#### (e) OLS

In [7]:
βOLS = (X'*X)\X'*Y

2-element Vector{Float64}:
 0.3456312177026701
 0.25026850544971313

#### (f) Monte Carlo

In [8]:
rep = 10000
biasProbit=0
biasLogit=0
biasOLS=0
rmseProbit=0
rmseLogit=0
rmseOLS=0
for i in 1:rep
    Random.seed!(i);
    (Y, X, U) = data(n)
    result = optimize(b->sampleCriterion(n,Y,b,X)[1,1],[0.0 ; 0.0])
    βProbit = Optim.minimizer(result)
    
    result = optimize(b->logitCriterion(n,Y,b,X)[1,1],[0.0 ; 0.0])
    βLogit = Optim.minimizer(result)
    
    βOLS = (X'*X)\X'*Y
    
    biasProbit += βProbit[2] -1;
    rmseProbit += (βProbit[2] -1)^2;
    biasLogit += βLogit[2] -1;
    rmseLogit += (βLogit[2] -1)^2
    biasOLS += βOLS[2] -1;
    rmseOLS += (βOLS[2] -1)^2;
end

biasProbit = biasProbit/rep
biasLogit = biasLogit/rep
biasOLS = biasOLS/rep
rmseProbit = sqrt(rmseProbit/rep)
rmseLogit = sqrt(rmseLogit/rep)
rmseOLS = sqrt(rmseOLS/rep)

table = [ "Bias" biasProbit biasLogit biasOLS;
"RMSE" rmseProbit rmseLogit rmseOLS]
pretty_table(table, ["" "Probit" "Logit" "OLS"])

┌──────┬───────────┬──────────┬───────────┐
│      │    Probit │    Logit │       OLS │
├──────┼───────────┼──────────┼───────────┤
│ Bias │ 0.0480279 │ 0.795611 │ -0.734105 │
│ RMSE │  0.228026 │ 0.891947 │   0.73481 │
└──────┴───────────┴──────────┴───────────┘


#### (g) Which is best?
Probit has a much smaller bias and RMSE than both Logit and OLS. 

#### (h) Do it again but bigger

In [9]:
n=1000
rep = 10000
biasProbit=0
biasLogit=0
biasOLS=0
rmseProbit=0
rmseLogit=0
rmseOLS=0
for i in 1:rep
    Random.seed!(i);
    (Y, X, U) = data(n)
    result = optimize(b->sampleCriterion(n,Y,b,X)[1,1],[0.0 ; 0.0])
    βProbit = Optim.minimizer(result)
    
    result = optimize(b->logitCriterion(n,Y,b,X)[1,1],[0.0 ; 0.0])
    βLogit = Optim.minimizer(result)
    
    βOLS = (X'*X)\X'*Y
    
    biasProbit += βProbit[2] -1;
    rmseProbit += (βProbit[2] -1)^2;
    biasLogit += βLogit[2] -1;
    rmseLogit += (βLogit[2] -1)^2
    biasOLS += βOLS[2] -1;
    rmseOLS += (βOLS[2] -1)^2;
end

biasProbit = biasProbit/rep
biasLogit = biasLogit/rep
biasOLS = biasOLS/rep
rmseProbit = sqrt(rmseProbit/rep)
rmseLogit = sqrt(rmseLogit/rep)
rmseOLS = sqrt(rmseOLS/rep)

table = [ "Bias" biasProbit biasLogit biasOLS;
"RMSE" rmseProbit rmseLogit rmseOLS]
pretty_table(table, ["" "Probit" "Logit" "OLS"])

┌──────┬────────────┬──────────┬───────────┐
│      │     Probit │    Logit │       OLS │
├──────┼────────────┼──────────┼───────────┤
│ Bias │ 0.00347562 │ 0.718006 │ -0.734958 │
│ RMSE │  0.0629207 │ 0.727009 │  0.735026 │
└──────┴────────────┴──────────┴───────────┘
